In [1]:
list1 = [0.1, 0.2, 0.4, 0.1]
list2 = [0.1, 0.3, 0.1, 0.3]

In [7]:
for p1, l1 in enumerate(list1):
    for p2, l2 in enumerate(list2):
        listX.append([])
        listX[p1].append([])
        listX[p1][p2] = 10
listX

[[10,
  10,
  10,
  10,
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []],
 [10, 10, 10, 10, [], [], [], [], [], [], [], []],
 [10, 10, 10, 10, [], [], [], [], [], [], [], []],
 [10, 10, 10, 10, [], [], [], [], [], [], [], []],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]